In [1]:
# ==============================================================================
# ЯЧЕЙКА 1: Импорты и основные конфигурационные параметры
# ==============================================================================
import os
import xml.etree.ElementTree as ET
import re
from xml.sax.saxutils import escape # Для экранирования текстового содержимого XML
from collections import defaultdict # Для удобного подсчета
import os
import xml.etree.ElementTree as ET
from transformers import MarianMTModel, MarianTokenizer
import torch
from xml.sax.saxutils import unescape, escape # Для работы с экранированным текстом
import time # Для индикатора прогресса

MODS_ROOT_DIRECTORY = r"C:\Users\user\AppData\Local\Daedalic Entertainment GmbH\Barotrauma\WorkshopMods\Installed" # <--- ИЗМЕНИТЬ ЗДЕСЬ

# Директория для сохранения результатов (будет создана относительно места запуска ноутбука,
# если не указан абсолютный путь)
OUTPUT_DIRECTORY_NAME = "translation_output_for_extractor"
OUTPUT_FILENAME = "strings_for_translation.xml"

# Язык, который мы хотим извлечь (исходный язык текстов из XML)
SOURCE_LANGUAGE_FILTER = "English"
# Язык, наличие которого в XML означает, что строка уже переведена и ее не нужно включать
EXISTING_TRANSLATION_LANGUAGE = "Russian"

# Язык, который будет указан в выходном XML-файле (целевой язык перевода)
TARGET_OUTPUT_LANGUAGE = "Russian" # Для атрибута language в итоговом файле
TARGET_OUTPUT_TRANSLATED_NAME = "Русский" # Для атрибута translatedname в итоговом файле

# Порог для вывода часто встречающихся тегов
DUPLICATE_TAG_THRESHOLD = 5

# Проверка, что путь к модам задан
if MODS_ROOT_DIRECTORY == "ПУТЬ_К_ВАШЕЙ_ПАПКЕ_С_МОДАМИ" or not os.path.isdir(MODS_ROOT_DIRECTORY):
    print(f"ОШИБКА: Пожалуйста, укажите корректный путь к папке с модами в переменной 'MODS_ROOT_DIRECTORY'.")
    print(f"Текущее значение: '{MODS_ROOT_DIRECTORY}'")
    # В Jupyter можно остановить выполнение ячейки через raise Exception или просто дать пользователю исправить
    # raise ValueError("Путь к модам не указан или указан неверно.")
else:
    print(f"Путь к модам установлен: {os.path.abspath(MODS_ROOT_DIRECTORY)}")

# Путь для сохранения выходного файла
FULL_OUTPUT_PATH = os.path.join(OUTPUT_DIRECTORY_NAME, OUTPUT_FILENAME)

C:\Users\user\miniforge3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Путь к модам установлен: C:\Users\user\AppData\Local\Daedalic Entertainment GmbH\Barotrauma\WorkshopMods\Installed


In [2]:
# ==============================================================================
# ЯЧЕЙКА 2: Вспомогательные функции
# ==============================================================================

def sanitize_xml_tag_name(name):
    """Санитизирует строку, чтобы она была валидным именем XML-тега."""
    if not isinstance(name, str):
        name = str(name)
    
    name = re.sub(r'\s+', '_', name)
    name = re.sub(r'[^a-zA-Z0-9_.-]', '', name)
    
    if not name:
        return "sanitized_empty_tag"
        
    if re.match(r'^[0-9.-]', name) or name.lower().startswith("xml"):
        name = "_" + name
    
    if not name: # Дополнительная проверка, если после добавления "_" имя стало пустым (маловероятно)
        return "invalid_tag_fallback"
    return name

def get_mod_name_from_path(filepath, base_mods_directory):
    """Определяет имя мода на основе пути к файлу и корневой директории модов."""
    try:
        normalized_filepath = os.path.normpath(filepath)
        normalized_base_mods_directory = os.path.normpath(base_mods_directory)
        
        # Убедимся, что base_mods_directory заканчивается разделителем пути,
        # чтобы relpath корректно отсекал его.
        if not normalized_base_mods_directory.endswith(os.sep):
            normalized_base_mods_directory += os.sep
            
        if not normalized_filepath.startswith(normalized_base_mods_directory):
            # Если файл находится вне ожидаемой структуры (например, base_mods_directory указан неверно)
            # Попробуем взять имя родительской папки файла как имя мода
            parent_dir = os.path.basename(os.path.dirname(normalized_filepath))
            # print(f"Warning: File {normalized_filepath} is outside base_mods_directory {normalized_base_mods_directory}. Using parent dir '{parent_dir}' as mod name.")
            return parent_dir if parent_dir else "UnknownModContext"

        relative_path = os.path.relpath(normalized_filepath, normalized_base_mods_directory)
        path_parts = relative_path.split(os.sep)
        
        # Имя мода - это первая часть относительного пути
        # Например, если base_mods_directory = /mods/ и filepath = /mods/MyMod/file.xml,
        # то relative_path = MyMod/file.xml, и path_parts[0] = MyMod.
        if path_parts and path_parts[0] and path_parts[0] not in ('.', '..'):
            return path_parts[0]
        else:
            # Это случай, когда файл находится прямо в base_mods_directory (не в подпапке мода).
            # Или если relpath вернул что-то неожиданное.
            # print(f"Warning: Could not determine mod name for {normalized_filepath} within {normalized_base_mods_directory}. Using base directory name.")
            # Вернем имя самой папки base_mods_directory или ее части
            return os.path.basename(os.path.normpath(base_mods_directory)) # Используем normpath, чтобы убрать возможный последний /

    except ValueError: 
        # Может возникнуть, если пути на разных дисках в Windows (os.path.relpath)
        # В этом случае, просто берем имя родительской папки файла
        parent_dir = os.path.basename(os.path.dirname(normalized_filepath))
        # print(f"ValueError determining mod name for {filepath}. Using parent dir '{parent_dir}'.")
        return parent_dir if parent_dir else "UnknownModPathError"


def extract_keys_from_xml(filepath, lang_to_extract):
    """Извлекает ключи (санитизированные full_tag) из XML файла для указанного языка."""
    keys = set()
    try:
        tree = ET.parse(filepath)
        root = tree.getroot()
        file_language = root.get("language")

        if file_language and file_language.lower() == lang_to_extract.lower():
            for element in root.iter():
                # Исключаем корневой тег infotexts и тег style, если он есть
                if element.tag.lower() in ["infotexts", "style"]:
                    continue
                
                original_tag_name = element.tag
                # Попытка получить 'identifier' или 'name' для более уникального тега
                id_val = element.get('identifier') or element.get('name')
                
                sanitized_main_tag = sanitize_xml_tag_name(original_tag_name)
                if id_val:
                    sanitized_id_val = sanitize_xml_tag_name(id_val)
                    # Убедимся, что sanitized_id_val действительно что-то содержит и не является просто "меткой" ошибки
                    if sanitized_id_val and sanitized_id_val not in ("sanitized_empty_tag", "invalid_tag_fallback"):
                        full_tag = f"{sanitized_main_tag}.{sanitized_id_val}"
                    else:
                        full_tag = sanitized_main_tag # Возвращаемся к использованию только основного тега
                else:
                    full_tag = sanitized_main_tag
                
                full_tag = sanitize_xml_tag_name(full_tag) # Финальная санация всего полного тега
                keys.add(full_tag)
        return keys
    except ET.ParseError:
        # print(f"XML Parse Error (extract_keys_from_xml) in file {filepath}. Skipping.")
        return set()
    except Exception as e:
        # print(f"Error (extract_keys_from_xml) in file {filepath}: {e}. Skipping.")
        return set()


def extract_text_from_xml_file(filepath, base_mods_directory, lang_filter):
    """Извлекает (tag, text, path, mod) из XML, если он соответствует языковому фильтру."""
    text_list_for_file = []
    try:
        tree = ET.parse(filepath)
        root = tree.getroot()
        file_language = root.get("language")

        process_this_file = False
        # Логика определения, нужно ли обрабатывать файл:
        # 1. Если язык файла явно указан и совпадает с lang_filter.
        # 2. Если язык файла не указан, но lang_filter установлен в "english" (по умолчанию для многих игр).
        if file_language:
            if file_language.lower() == lang_filter.lower():
                process_this_file = True
        elif lang_filter.lower() == "english": # Предполагаем, что файлы без атрибута language - английские
            process_this_file = True
        
        if not process_this_file:
            return []

        mod_name = get_mod_name_from_path(filepath, base_mods_directory)

        excluded_tags = {
            "infotexts", "style", "sound", "sprite", "animation", "limb", "trigger",
            "statvalue", "objective", "particleemitter", "damagemodifier", "attack",
            "character", "job", "item", "structure", "locationtype",
            "levelgenerationparameters", "mission", "event", "eventset", "characterinfo",
            "ragdoll", "campaignsettings", "destructible", "fabricator", "deconstructor",
            "repairable", "controller", "connectionpanel", "engine", "pump", "reactor",
            "turret", "itemcontainer", "door", "medicalclinic", "talenttree", "talents",
            "submarine", "shuttle", "upgradecategory", "upgrademodule", "afflictions",
            "geneticmaterial", "mapgenerationparameters", "allowwhenriding", "allowatsub",
            "allowatbeaconstation", "allowatoutpost", "allowatcity", "allowatcolonies",
            "allowatdestroyeddoutpost", "allowatabandonedoutpost", "allowatruins",
            "allowatwreck", "allowatcave", "allowatpirateoutpost", "commonness",
            "requiredcampaignlevel", "campaignonly", "health", "price", "fabricationtime", 
            "deconstructtime", "containable", "spritecolor", "decorativesprite", "music",
            "useverb", "examineverb", "pickupverb", 
            "requireditem", "requiredskill", "itemidentifier", "structureidentifier",
            "characteridentifier", "soundfile", "musicfile", "imagefile", "texture", "animationfile",
            "soundchannel", "soundvolume", "soundrange", "loop", "playonstart",
            "color", "vector2", "vector3", "vector4", "rect", "point", "offset", "scale", "size",
            "limbname", "bonename", "jointname", 
            "state", "type", "category", "group", "layer", "order", "slot",
            "targettag", "sourcetag", "linkedsub", "linkeduuid",
            "variable", "property", "value", 
            "button", 
            "command", "script", "function", "eventname",
            "dialogflag", "objectiveflag", "questflag", 
            "classname", "speciesname", 
            "filename", "path", 
            "default", 
            "ambientmccormicks", 'returns', 'remarks', 'c', 'para', 'see', 'param.il', 'param.steamid', 
            'param.appid', 'param.name', 'code', 'param.filename', 'param.type', 'param.character', 
            'param.frequency', 'param.sampleRate', 'param.action', 'param.identifier', 
            'param.interactableFor', 'param.statName', 'param.value', 'param.position', 
            'param.assembly', 'param.createNetworkEvent', 'param.defult', 'param.force', 
            'param.load', 'param.predicate', 'param.prefab', 'param.radius', 'typeparam.T', 
            'exception', 'override', 'locationchange.base.changeto.military', 
            'eventtext.blockadealarm.breakin', 'locationnameformat.mine', 'loadingscreentip', 
            'dialogturnoffsonar', 'dialogcantfindanechoicsuit', 'lua_name', 'lua_description', 
            "author", "id", "param.createNetworkEvent", 'summary'
        }


        for element in root.iter():
            # Пропускаем теги из списка исключений
            if element.tag.lower() in excluded_tags:
                continue

            if element.text: # Убеждаемся, что у элемента есть текстовое содержимое
                original_tag_name = element.tag
                id_val = element.get('identifier') or element.get('name')
                
                sanitized_main_tag = sanitize_xml_tag_name(original_tag_name)
                if id_val:
                    sanitized_id_val = sanitize_xml_tag_name(id_val)
                    if sanitized_id_val and sanitized_id_val not in ("sanitized_empty_tag", "invalid_tag_fallback"):
                        full_tag = f"{sanitized_main_tag}.{sanitized_id_val}"
                    else:
                        full_tag = sanitized_main_tag
                else:
                    full_tag = sanitized_main_tag
                
                full_tag = sanitize_xml_tag_name(full_tag) # Финальная санация

                stripped_text = element.text.strip()
                if stripped_text: # Только если текст не пустой после удаления пробелов
                    escaped_text_content = escape(stripped_text)
                    text_list_for_file.append((full_tag, escaped_text_content, filepath, mod_name))
        return text_list_for_file
    except ET.ParseError:
        print(f"XML Parse Error processing file {filepath}. Skipping.")
        return []
    except Exception as e:
        print(f"Error processing XML file {filepath}: {e}")
        return []

def extract_text_from_lua_file(filepath, base_mods_directory):
    """Извлекает тексты из Lua файлов."""
    text_list_for_file = []
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            content = f.read()
        mod_name = get_mod_name_from_path(filepath, base_mods_directory)
        
        # Регулярное выражение для поиска name = "...", description = "...", etc.
        # (?i) - регистронезависимый поиск ключа
        # \b(name|label|...)\b - слово целиком (ключ)
        # \s*=\s* - равно с пробелами вокруг
        # "([^"]*)" - строка в двойных кавычках, захватываем содержимое
        kv_pairs = re.findall(r'(?i)\b(name|label|displayname|tooltip|description|text|caption|title|message)\s*=\s*"([^"]*)"', content)
        for key, value in kv_pairs:
            clean_value = value.strip()
            if clean_value: # Только если значение не пустое
                # Создаем тег на основе ключа из Lua
                lua_tag = sanitize_xml_tag_name(f"lua_{key.lower()}")
                escaped_value = escape(clean_value)
                text_list_for_file.append((lua_tag, escaped_value, filepath, mod_name))

        # Регулярное выражение для поиска вызовов функций типа Text("..."), Texts.Get("..."), Game.ShowMessageBox("...")
        # (?i) - регистронезависимый поиск
        # \b(?:Text|Texts\.Get|Game\.ShowMessageBox)\b - известные функции, работающие с текстом
        # \s*\(\s* - открывающая скобка с пробелами
        # "([^"]*)" - строка в двойных кавычках
        text_func_calls = re.findall(r'(?i)\b(?:Text|Texts\.Get|Game\.ShowMessageBox)\s*\(\s*"([^"]*)"', content)
        for text_val in text_func_calls:
            clean_value = text_val.strip()
            if clean_value: # Только если значение не пустое
                lua_tag = sanitize_xml_tag_name("lua_func_text") # более общий тег для текстов из функций
                escaped_value = escape(clean_value)
                text_list_for_file.append((lua_tag, escaped_value, filepath, mod_name))
        
        return text_list_for_file
    except Exception as e:
        print(f"Error processing Lua file {filepath}: {e}")
        return []

In [3]:
# ==============================================================================
# ЯЧЕЙКА 3: Основные функции сбора и сохранения текста
# ==============================================================================

def collect_and_filter_texts(mods_root_directory):
    """Собирает все тексты, фильтрует по языку, исключает переведенные, дедуплицирует."""
    
    # Словарь для хранения уже переведенных ключей XML по модам
    # { "mod_name": {"key1", "key2"}, ... }
    translated_xml_keys_by_mod = defaultdict(set)
    
    print(f"Phase 1: Scanning for existing XML translations in '{EXISTING_TRANSLATION_LANGUAGE}'...")
    xml_files_count_phase1 = 0
    for root_dir_scanned, _, files in os.walk(mods_root_directory):
        for file in files:
            if file.endswith(".xml"):
                xml_files_count_phase1 +=1
                filepath = os.path.join(root_dir_scanned, file)
                # Определяем имя мода для текущего файла
                mod_name_for_keys = get_mod_name_from_path(filepath, mods_root_directory)
                
                # Извлекаем ключи из XML-файла, если он на языке EXISTING_TRANSLATION_LANGUAGE
                keys_from_file = extract_keys_from_xml(filepath, EXISTING_TRANSLATION_LANGUAGE)
                if keys_from_file:
                    translated_xml_keys_by_mod[mod_name_for_keys].update(keys_from_file)

    total_translated_keys = sum(len(s) for s in translated_xml_keys_by_mod.values())
    print(f"Scanned {xml_files_count_phase1} XML files. Found {total_translated_keys} XML tags in {len(translated_xml_keys_by_mod)} mods already translated to '{EXISTING_TRANSLATION_LANGUAGE}'.")

    # Список для всех текстов, которые нужно будет перевести
    all_source_texts_to_translate = []
    # Множество для дедупликации текстов (mod_name, full_tag, escaped_original_text)
    seen_global_text_keys_for_dedup = set()

    # Словари для сбора статистики по тегам
    # (mod_name, full_tag) -> count
    tag_occurrences = defaultdict(int)
    # (mod_name, full_tag) -> set of (escaped_text, original_filepath)
    tag_details_map = defaultdict(set)

    print(f"\nPhase 2: Scanning for source texts ('{SOURCE_LANGUAGE_FILTER}' XML & Lua), filtering and deduplicating...")
    processed_files_count_phase2 = 0
    
    for root_dir_scanned, _, files in os.walk(mods_root_directory):
        for file in files:
            filepath = os.path.join(root_dir_scanned, file)
            current_file_source_texts = [] # Тексты из текущего файла
            is_lua_file = False

            if file.endswith(".xml"):
                processed_files_count_phase2 += 1
                current_file_source_texts = extract_text_from_xml_file(filepath, mods_root_directory, SOURCE_LANGUAGE_FILTER)
            elif file.endswith(".lua"):
                processed_files_count_phase2 += 1
                is_lua_file = True
                current_file_source_texts = extract_text_from_lua_file(filepath, mods_root_directory)

            # Обработка текстов, извлеченных из текущего файла
            for full_tag, escaped_original_text, source_filepath, mod_name in current_file_source_texts:
                # Сбор статистики для анализа частоты тегов
                tag_key_for_stats = (mod_name, full_tag)
                tag_occurrences[tag_key_for_stats] += 1
                tag_details_map[tag_key_for_stats].add((escaped_original_text, os.path.normpath(source_filepath)))

                # Проверка, был ли этот XML-тег уже переведен в данном моде
                is_already_translated_in_mod = False
                if not is_lua_file: # Проверка на перевод актуальна только для XML
                    if mod_name in translated_xml_keys_by_mod and \
                       full_tag in translated_xml_keys_by_mod[mod_name]:
                        is_already_translated_in_mod = True
                
                # Ключ для глобальной дедупликации (мод, тег, текст)
                text_key_for_dedup = (mod_name, full_tag, escaped_original_text)
                
                if text_key_for_dedup not in seen_global_text_keys_for_dedup and not is_already_translated_in_mod:
                    seen_global_text_keys_for_dedup.add(text_key_for_dedup)
                    all_source_texts_to_translate.append((full_tag, escaped_original_text, source_filepath, mod_name))

    print(f"Processed {processed_files_count_phase2} XML/Lua files for source text.")
    
    # Сортировка для консистентного вывода (сначала по имени мода, потом по тегу, потом по тексту)
    all_source_texts_to_translate.sort(key=lambda x: (x[3].lower(), x[0].lower(), x[1].lower()))
    
    # Анализ и формирование информации о часто встречающихся тегах
    frequent_tags_report = []
    for (mod_name, tag_name), count in tag_occurrences.items():
        if count >= DUPLICATE_TAG_THRESHOLD:
            details = tag_details_map[(mod_name, tag_name)]
            unique_texts_in_tag = {text for text, path in details}
            unique_filepaths_in_tag = {path for text, path in details}
            
            frequent_tags_report.append({
                "mod_name": mod_name,
                "tag_name": tag_name,
                "count": count,
                "unique_texts": unique_texts_in_tag,
                "unique_filepaths": unique_filepaths_in_tag,
                "sample_details": list(details)[:min(3, len(details))] # Первые 3 примера (текст, путь)
            })
    
    # Сортировка отчета по частоте тегов (по моду, по убыванию количества, по имени тега)
    frequent_tags_report.sort(key=lambda x: (x["mod_name"].lower(), -x["count"], x["tag_name"].lower()))

    return all_source_texts_to_translate, frequent_tags_report


def save_texts_to_final_xml(text_items_list, output_filepath, lang_attr, translated_name_attr):
    """Сохраняет собранные и отфильтрованные тексты в итоговый XML."""
    output_dir = os.path.dirname(output_filepath)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    with open(output_filepath, "w", encoding="utf-8") as f:
        f.write(f'<?xml version="1.0" encoding="utf-8"?>\n')
        f.write(f'<infotexts language="{lang_attr}" nowhitespace="false" translatedname="{translated_name_attr}">\n\n')
        
        current_mod_for_comment = None
        for full_tag, escaped_original_text_content, source_filepath, mod_name in text_items_list:
            # Добавляем комментарий с именем мода при его смене
            if mod_name != current_mod_for_comment:
                if current_mod_for_comment is not None: # Добавляем пустую строку перед новым блоком мода
                    f.write('\n') 
                f.write(f'  <!-- Texts from Mod: {mod_name} -->\n')
                current_mod_for_comment = mod_name
            
            # Добавляем комментарий с путем к оригинальному файлу
            f.write(f'  <!-- Original File: {os.path.normpath(source_filepath)} -->\n')
            # Записываем сам тег и его содержимое
            f.write(f'  <{full_tag}>{escaped_original_text_content}</{full_tag}>\n')
        
        f.write('\n</infotexts>\n')

In [4]:
# ==============================================================================
# ЯЧЕЙКА 4: Запуск процесса извлечения и сохранения
# ==============================================================================

# Убедимся, что путь к модам задан корректно перед запуском
if MODS_ROOT_DIRECTORY == "ПУТЬ_К_ВАШЕЙ_ПАПКЕ_С_МОДАМИ" or not os.path.isdir(MODS_ROOT_DIRECTORY):
    print(f"ОШИБКА: Запуск невозможен. Укажите корректный путь к папке с модами в переменной 'MODS_ROOT_DIRECTORY' в ЯЧЕЙКЕ 1.")
else:
    print(f"--- Starting Text Extraction Script ---")
    print(f"Mods directory: {os.path.abspath(MODS_ROOT_DIRECTORY)}")
    print(f"Source language (XML): '{SOURCE_LANGUAGE_FILTER}'")
    print(f"Excluding XML texts if already translated to '{EXISTING_TRANSLATION_LANGUAGE}' (within the same mod).")
    print(f"Output will be prepared for target language '{TARGET_OUTPUT_LANGUAGE}'.")
    print(f"Threshold for reporting frequent tags: {DUPLICATE_TAG_THRESHOLD} occurrences per mod.")
    print(f"Output file will be: {os.path.abspath(FULL_OUTPUT_PATH)}")
    print(f"-----------------------------------------")
    
    final_texts_for_translation, frequent_tags_data = collect_and_filter_texts(MODS_ROOT_DIRECTORY)
    
    if final_texts_for_translation:
        print(f"\n--- Results: Texts for Translation ---")
        print(f"Found {len(final_texts_for_translation)} unique text entries requiring translation.")
        save_texts_to_final_xml(final_texts_for_translation, FULL_OUTPUT_PATH, TARGET_OUTPUT_LANGUAGE, TARGET_OUTPUT_TRANSLATED_NAME)
        print(f"Output file saved to: {os.path.abspath(FULL_OUTPUT_PATH)}")
    else:
        print(f"\n--- Results: Texts for Translation ---")
        print(f"No new texts found needing translation based on the specified criteria.")
    
    if frequent_tags_data:
        print(f"\n--- Frequent Tags Analysis (>= {DUPLICATE_TAG_THRESHOLD} occurrences per tag per mod) ---")
        print(f"Found {len(frequent_tags_data)} tag types that appear frequently. ")
        print(f"Review these tags. If their content is not meant for translation or is redundant,")
        print(f"consider adding the original XML tag name (before sanitization) to the 'excluded_tags' list ")
        print(f"in the 'extract_text_from_xml_file' function, or adjust Lua parsing if needed.")
        
        for tag_info in frequent_tags_data:
            print(f"\n  Mod: {tag_info['mod_name']}")
            print(f"    Tag (sanitized): '{tag_info['tag_name']}'")
            print(f"    Occurrences: {tag_info['count']}")
            
            unique_texts = list(tag_info['unique_texts'])
            if len(unique_texts) == 1:
                print(f"    Associated Text (consistent): \"{unique_texts[0]}\"")
            else:
                print(f"    Associated Texts ({len(unique_texts)} unique variants, showing up to 3):")
                for i, text_sample in enumerate(unique_texts[:3]):
                    print(f"      - \"{text_sample}\"")
                if len(unique_texts) > 3:
                    print(f"      ... and {len(unique_texts) - 3} more variants.")
            
            print(f"    Found in {len(tag_info['unique_filepaths'])} unique files. Examples of (text, file):")
            for text_ex, file_ex in tag_info['sample_details']:
                 print(f"      - \"{text_ex}\" (from: {file_ex})")

    else:
        print(f"\n--- Frequent Tags Analysis ---")
        print(f"No tags met the frequency threshold of {DUPLICATE_TAG_THRESHOLD} occurrences per mod.")
        
    print(f"-----------------------------------------")
    print(f"Script finished.")

--- Starting Text Extraction Script ---
Mods directory: C:\Users\user\AppData\Local\Daedalic Entertainment GmbH\Barotrauma\WorkshopMods\Installed
Source language (XML): 'English'
Excluding XML texts if already translated to 'Russian' (within the same mod).
Output will be prepared for target language 'Russian'.
Threshold for reporting frequent tags: 5 occurrences per mod.
Output file will be: E:\coding\!Jupetr\Barotrauma\translation_output_for_extractor\strings_for_translation.xml
-----------------------------------------
Phase 1: Scanning for existing XML translations in 'Russian'...
Scanned 3822 XML files. Found 29105 XML tags in 29 mods already translated to 'Russian'.

Phase 2: Scanning for source texts ('English' XML & Lua), filtering and deduplicating...
XML Parse Error processing file C:\Users\user\AppData\Local\Daedalic Entertainment GmbH\Barotrauma\WorkshopMods\Installed\3012187347\模組更新紀錄(自用).xml. Skipping.
Processed 3998 XML/Lua files for source text.

--- Results: Texts for T

In [5]:
# --- Конфигурация ---
INPUT_XML_FILE = "translation_output_for_extractor/strings_for_translation.xml"
OUTPUT_XML_FILE_TRANSLATED = "translation_output_final/translated_with_inline_originals.xml"

MODEL_NAME = 'Helsinki-NLP/opus-mt-en-ru'
TARGET_LANGUAGE_CODE_ATTR = "Russian"
TARGET_TRANSLATED_NAME_ATTR = "Русский"

ATTEMPT_MODEL_TRANSLATION = True # True для перевода, False для "Русский (Оригинал)"

# Разделитель между переводом и оригиналом в тексте элемента
TEXT_SEPARATOR = "\n---\n" # \n добавит перенос строки до и после

# Определяем устройство (CPU или GPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Создадим директории, если их нет, для примерного XML
os.makedirs(os.path.dirname(INPUT_XML_FILE), exist_ok=True)
os.makedirs(os.path.dirname(OUTPUT_XML_FILE_TRANSLATED), exist_ok=True)

# Примерный XML, если вы запускаете впервые и файла нет
# В реальном сценарии вы будете использовать свой файл
if not os.path.exists(INPUT_XML_FILE):
    example_xml_content = """<?xml version="1.0" encoding="UTF-8"?>
<root language="English" translatedname="Английский">
  <string id="1">Hello, world!</string>
  <string id="2">This is a test string.</string>
  <nested>
    <item>Another example of text & entities.</item>
  </nested>
  <!-- Это комментарий, он будет проигнорирован -->
  <infoTexts>This text should be ignored.</infoTexts>
  <style>This style information should be ignored.</style>
  <emptyElement></emptyElement>
  <elementWithSpacesOnly>   </elementWithSpacesOnly>
</root>
"""
    with open(INPUT_XML_FILE, 'w', encoding='utf-8') as f:
        f.write(example_xml_content)
    print(f"Created a sample input file: {INPUT_XML_FILE}")

Using device: cuda


In [6]:
def load_model_and_tokenizer(model_name):
    print(f"Loading tokenizer for {model_name}...")
    try:
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        print(f"Loading model {model_name}...")
        model = MarianMTModel.from_pretrained(model_name)
        model.to(DEVICE)
        model.eval() # Переводим модель в режим оценки (важно для инференса)
        print("Model and tokenizer loaded.")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model/tokenizer: {e}")
        print("Ensure you have an internet connection for the first download, or the model is cached.")
        print("Try: pip install sentencepiece sacremoses")
        return None, None

def translate_texts_batch(texts_to_translate, model, tokenizer, batch_size=8):
    if not model or not tokenizer:
        print("Model or tokenizer not loaded, skipping translation.")
        return [f"[MODEL_NOT_LOADED] {text}" for text in texts_to_translate]

    translations = []
    total_texts = len(texts_to_translate)
    print(f"Starting translation for {total_texts} texts with batch_size={batch_size}...")
    start_time_total = time.time()

    for i in range(0, total_texts, batch_size):
        batch_original_texts = texts_to_translate[i:i+batch_size]
        
        # Пропускаем батч, если все строки в нем пустые или состоят из пробелов
        if not any(t.strip() for t in batch_original_texts):
            translations.extend([""] * len(batch_original_texts)) # Добавляем пустые строки для пустых оригиналов
            # Обновление прогресса для последнего батча, если он был пропущен
            if i + batch_size >= total_texts:
                 elapsed_total = time.time() - start_time_total
                 print(f"  Progress: {total_texts}/{total_texts} (100.00%) | Total time: {elapsed_total:.2f}s (Skipped empty batch)")
            continue

        try:
            start_time_batch = time.time()
            # Токенизируем батч
            tokenized_batch = tokenizer(batch_original_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(DEVICE)
            
            # Генерируем перевод
            with torch.no_grad(): # Отключаем вычисление градиентов для ускорения и экономии памяти
                translated_tokens = model.generate(**tokenized_batch)
            
            # Декодируем токены обратно в текст
            batch_translations = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
            translations.extend(batch_translations)
            
            # Индикатор прогресса
            processed_count = min(i + batch_size, total_texts)
            percentage = (processed_count / total_texts) * 100
            elapsed_batch = time.time() - start_time_batch
            elapsed_total = time.time() - start_time_total
            
            print(f"  Progress: {processed_count}/{total_texts} ({percentage:.2f}%) | Batch time: {elapsed_batch:.2f}s | Total time: {elapsed_total:.2f}s")

        except Exception as e:
            print(f"Error translating batch starting with '{batch_original_texts[0][:30]}...': {e}")
            translations.extend([f"[TRANSLATION_ERROR] {text}" for text in batch_original_texts])
            # Обновляем прогресс даже при ошибке
            processed_count = min(i + batch_size, total_texts)
            percentage = (processed_count / total_texts) * 100
            elapsed_total = time.time() - start_time_total
            print(f"  Progress: {processed_count}/{total_texts} ({percentage:.2f}%) | ERROR IN BATCH | Total time: {elapsed_total:.2f}s")

    print(f"Translation finished for {total_texts} texts. Total time: {time.time() - start_time_total:.2f}s")
    return translations

In [7]:
model, tokenizer = None, None
if ATTEMPT_MODEL_TRANSLATION:
    model, tokenizer = load_model_and_tokenizer(MODEL_NAME)
    if not model or not tokenizer:
        print("Failed to load model. Translation will be skipped, structure will be 'Original Text [SEPARATOR] Original Text'.")
else:
    print("Model translation is disabled by configuration (ATTEMPT_MODEL_TRANSLATION=False).")

Loading tokenizer for Helsinki-NLP/opus-mt-en-ru...
Loading model Helsinki-NLP/opus-mt-en-ru...
Model and tokenizer loaded.


In [8]:
      
try:
    tree = ET.parse(INPUT_XML_FILE)
    root = tree.getroot()
    print(f"Successfully parsed XML file: {INPUT_XML_FILE}")
except FileNotFoundError:
    print(f"Error: Input file not found at {INPUT_XML_FILE}")
    # Остановить выполнение, если файл не найден, или обработать иначе
    raise 
except ET.ParseError as e:
    print(f"Error: Could not parse XML file {INPUT_XML_FILE}: {e}")
    # Остановить выполнение или обработать
    raise

    

Successfully parsed XML file: translation_output_for_extractor/strings_for_translation.xml


In [9]:
original_texts_unescaped = [] # Расэкранированные оригиналы
elements_to_update = []       # XML элементы для обновления

print("Extracting texts from input XML...")
# Итерация по всем элементам в дереве
for element in root.iter():
    # Пропускаем комментарии, специальные теги и элементы без текста
    is_comment_node = isinstance(element.tag, type(ET.Comment)) # Проверка, является ли элемент комментарием
    if is_comment_node or element.tag.lower() in ["infotexts", "style"] or not element.text:
        continue

    escaped_text_from_input_xml = element.text.strip() # Получаем текст и убираем пробелы по краям
    
    if escaped_text_from_input_xml: # Если текст не пустой после strip()
        unescaped_original = unescape(escaped_text_from_input_xml) # Расэкранируем HTML-сущности
        original_texts_unescaped.append(unescaped_original)
        elements_to_update.append(element) # Сохраняем сам элемент для последующего обновления

if not original_texts_unescaped:
    print("No text entries found to process in the XML.")
else:
    print(f"Found {len(original_texts_unescaped)} text entries to process.")
    # Можно посмотреть несколько первых извлеченных текстов:
    # print("First few texts to translate:", original_texts_unescaped[:3])

Extracting texts from input XML...
Found 7411 text entries to process.


In [10]:
translated_or_marked_texts = []

if original_texts_unescaped: # Только если есть что переводить
    if ATTEMPT_MODEL_TRANSLATION and model and tokenizer:
        # Устанавливаем batch_size в зависимости от устройства
        # Для CPU лучше меньше, для GPU можно больше (зависит от VRAM)
        current_batch_size = 8 if DEVICE.type == 'cpu' else 16 # Примерные значения
        
        translated_results = translate_texts_batch(original_texts_unescaped, model, tokenizer, batch_size=current_batch_size)
        
        if len(translated_results) == len(original_texts_unescaped):
            translated_or_marked_texts = translated_results
        else:
            print("Error: Mismatch in count of translated texts. Using originals as fallback for structure.")
            # В случае ошибки, используем оригиналы, чтобы не сломать структуру
            translated_or_marked_texts = original_texts_unescaped 
    else:
        if ATTEMPT_MODEL_TRANSLATION and (not model or not tokenizer):
            print("Model translation was requested but model/tokenizer failed to load. Using original texts for structure.")
        elif not ATTEMPT_MODEL_TRANSLATION:
            print("Model translation is disabled. Using original texts for structure.")
        else: # original_texts_unescaped был пуст
             print("No texts were extracted, skipping translation step.")
        # Если перевод не нужен или не удался, используем оригиналы
        translated_or_marked_texts = original_texts_unescaped
        
    # Можно посмотреть несколько первых результатов перевода (или оригиналов, если перевод не делался)
    # if translated_or_marked_texts:
    #    print("First few processed texts:", translated_or_marked_texts[:3])
else:
    print("No texts were extracted, skipping translation step.")

Starting translation for 7411 texts with batch_size=16...
  Progress: 16/7411 (0.22%) | Batch time: 0.90s | Total time: 0.90s
  Progress: 32/7411 (0.43%) | Batch time: 3.71s | Total time: 4.61s
  Progress: 48/7411 (0.65%) | Batch time: 0.23s | Total time: 4.84s
  Progress: 64/7411 (0.86%) | Batch time: 0.13s | Total time: 4.97s
  Progress: 80/7411 (1.08%) | Batch time: 6.12s | Total time: 11.09s
  Progress: 96/7411 (1.30%) | Batch time: 0.42s | Total time: 11.51s
  Progress: 112/7411 (1.51%) | Batch time: 0.15s | Total time: 11.66s
  Progress: 128/7411 (1.73%) | Batch time: 0.15s | Total time: 11.82s
  Progress: 144/7411 (1.94%) | Batch time: 0.15s | Total time: 11.97s
  Progress: 160/7411 (2.16%) | Batch time: 0.16s | Total time: 12.13s
  Progress: 176/7411 (2.37%) | Batch time: 0.25s | Total time: 12.38s
  Progress: 192/7411 (2.59%) | Batch time: 0.16s | Total time: 12.54s
  Progress: 208/7411 (2.81%) | Batch time: 0.38s | Total time: 12.92s
  Progress: 224/7411 (3.02%) | Batch time:

In [11]:
if original_texts_unescaped and elements_to_update: # Только если были тексты и элементы для обновления
    print("Updating XML tree with combined translated/original text...")
    if len(translated_or_marked_texts) == len(elements_to_update) == len(original_texts_unescaped):
        for i, element_node in enumerate(elements_to_update):
            original_text = original_texts_unescaped[i]
            processed_text = translated_or_marked_texts[i] 

            # Формируем комбинированный текст
            # Если перевод пуст или содержит маркер ошибки, или если перевод = оригиналу,
            # можно решить не добавлять оригинал или добавить специальную пометку.
            # Текущая логика просто комбинирует.
            if not processed_text.strip() or "[TRANSLATION_ERROR]" in processed_text or "[MODEL_NOT_LOADED]" in processed_text:
                # Если перевод неудачен или пуст, можно просто оставить оригинал
                # или специальную пометку. Для данного примера, используем структуру с сепаратором.
                # processed_text = f"[FAILED_TRANSLATION] {original_text}" # Пример
                # В вашем коде вы уже обрабатываете это в translate_texts_batch,
                # так что processed_text уже будет содержать маркер ошибки.
                pass # processed_text уже содержит маркер ошибки

            combined_text = f"{processed_text}{TEXT_SEPARATOR}{original_text}"
            
            # Экранируем финальный комбинированный текст для вставки в XML
            escaped_combined_text = escape(combined_text)
            
            element_node.text = escaped_combined_text
            
            # Очищаем старые дочерние узлы (например, комментарии или другие текстовые узлы внутри), 
            # если они были и если это необходимо.
            # В данном случае, мы только обновляем .text, так что это может быть не нужно,
            # если внутри <element> нет других узлов, которые мы хотим удалить.
            # Но если там были, например, комментарии <element><!-- com -->text</element>,
            # то .text обновит "text", а комментарий останется.
            # Для полной очистки перед установкой нового текста:
            # for child in list(element_node):
            #      element_node.remove(child) # Это удалит ВСЕ дочерние элементы, если они есть
                                            # Будьте осторожны, если у вас сложная структура!
                                            # Если вы хотите сохранить дочерние *элементы*, но удалить только
                                            # внутренние текстовые узлы или комментарии, подход должен быть сложнее.
                                            # В вашем исходном коде вы удаляли все дочерние узлы.
            for child in list(element_node): # Как в вашем коде
                 element_node.remove(child)


        # Обновляем атрибуты корневого элемента
        root.set('language', TARGET_LANGUAGE_CODE_ATTR)
        root.set('translatedname', TARGET_TRANSLATED_NAME_ATTR)
        if 'nowhitespace' not in root.attrib: # Добавляем, только если его нет
            root.set('nowhitespace', "false")

        # Сохраняем измененное дерево
        try:
            # Для "красивой" печати XML (отступы) на Python 3.9+
            # tree.write автоматически обрабатывает xml_declaration, если он есть в дереве
            if hasattr(ET, 'indent'):
                ET.indent(tree) # Это изменяет tree "in-place"
            
            tree.write(OUTPUT_XML_FILE_TRANSLATED, encoding="utf-8", xml_declaration=True)
            print(f"Processed XML with inline originals saved to: {os.path.abspath(OUTPUT_XML_FILE_TRANSLATED)}")
        except Exception as e:
            print(f"Error saving XML file: {e}")
            if "ET.indent" in str(e) and not hasattr(ET, 'indent'):
                 print("Note: ET.indent(tree) for pretty printing is available in Python 3.9+. Try commenting it out if you use an older version or update Python.")

    else:
        print("Error: Mismatch in array lengths during XML update. Aborting XML update.")
        if not original_texts_unescaped:
             print("Reason: No original texts were extracted or available for processing.")
        elif not elements_to_update:
             print("Reason: No XML elements were marked for update.")
        else:
             print(f"Details: original_texts_unescaped: {len(original_texts_unescaped)}, elements_to_update: {len(elements_to_update)}, translated_or_marked_texts: {len(translated_or_marked_texts)}")

elif not original_texts_unescaped:
    print("No texts were extracted from XML. Nothing to update or save.")
    # Если исходный XML был пуст или не содержал текстов для перевода,
    # можно просто скопировать его или создать пустой выходной файл.
    # В данном случае, мы ничего не делаем, так как нечего обрабатывать.
else:
    print("Unexpected state: original_texts_unescaped has content, but elements_to_update is empty. This should not happen if extraction was successful.")

Updating XML tree with combined translated/original text...
Processed XML with inline originals saved to: E:\coding\!Jupetr\Barotrauma\translation_output_final\translated_with_inline_originals.xml


In [12]:
# Показать первые N строк из созданного файла
if os.path.exists(OUTPUT_XML_FILE_TRANSLATED):
    with open(OUTPUT_XML_FILE_TRANSLATED, 'r', encoding='utf-8') as f:
        print(f"\nFirst 15 lines of {OUTPUT_XML_FILE_TRANSLATED}:\n---")
        for i, line in enumerate(f):
            if i < 15:
                print(line, end='')
            else:
                break
        print("---")
else:
    print(f"Output file {OUTPUT_XML_FILE_TRANSLATED} was not created.")


First 15 lines of translation_output_final/translated_with_inline_originals.xml:
---
<?xml version='1.0' encoding='utf-8'?>
<infotexts language="Russian" nowhitespace="false" translatedname="Русский">
  <upgradecategory.cameras>Камеры
---
Cameras</upgradecategory.cameras>
  <upgradecategory.dischargecoils>Разрядка катушек
---
Discharge Coils</upgradecategory.dischargecoils>
  <upgradecategory.searchlights>Прожекторы
---
Searchlights</upgradecategory.searchlights>
  <upgradedescription.cameraincreaseoffsetonselected>Увеличьте дальность видимости всех камер.
---
Increase the view range of all cameras.</upgradedescription.cameraincreaseoffsetonselected>
  <upgradedescription.decreasenoiseproduction>Уменьшить поддающийся обнаружению шум, производимый этим устройством.
---


In [15]:
import os
from lxml import etree as ET
from xml.sax.saxutils import unescape, escape
import re
from tqdm import tqdm # Импортируем tqdm

# --- Конфигурация ---
INPUT_XML_FILE_TO_CLEAN = "translation_output_final/translated_with_inline_originals.xml"
OUTPUT_XML_FILE_CLEANED = "translation_output_final/translated_cleaned_lxml_tqdm.xml"
TEXT_SEPARATOR = "\n---\n"

# Обновленная и улучшенная функция post_process_translation
def post_process_translation(text):
    if not text:
        return ""
    
    # original_text_for_debug = text # Раскомментируйте для отладки и вывода в конце

    # --- 1. Нормализация пробелов ---
    # Заменяем все последовательности пробелов (включая различные виды пробелов) на один обычный пробел.
    text = re.sub(r'\s+', ' ', text).strip()

    # --- 2. Улучшенная обработка многоточий (и любых последовательностей точек) ---
    # НОВОЕ ПРАВИЛО: Заменяем ЛЮБУЮ последовательность из двух и более точек,
    # даже если они разделены пробелами, на уникальный плейсхолдер многоточия.
    # Это справится с "No. . . . ." -> "No___ELLIPSIS___", "команды. . . . ." -> "команды___ELLIPSIS___"
    # Паттерн `(\.\s*){2,}` означает: "точка, за которой следуют ноль или более пробелов,
    # повторяющаяся как минимум 2 раза".
    text = re.sub(r'(\.\s*){2,}', '___ELLIPSIS___', text)

    # --- 3. Очистка других знаков препинания (кроме точек, которые уже обработаны) ---

    # Коллапсируем множественные повторяющиеся восклицательные и вопросительные знаки.
    # Пример: "!!!", "???" -> "!", "?"
    text = re.sub(r'!{2,}', '!', text)
    text = re.sub(r'\?{2,}', '?', text)

    # Удаляем пробелы непосредственно перед знаками препинания (.,:;!?).
    # Точки (`.`) здесь оставлены, т.к. одиночная точка тоже должна "прилипать" к слову.
    text = re.sub(r'\s*([.,;:!?])', r'\1', text)

    # Гарантируем один пробел после знаков препинания (.,:;!? и ___ELLIPSIS___),
    # если за ними следует не знак препинания и не конец строки.
    # Это важно для "слово.слово" -> "слово. слово", но не для "слово.?"
    # Регулярное выражение:
    # ([.,;:!?]|___ELLIPSIS___): Захватывает знак препинания или плейсхолдер.
    # (?=\S): Позитивный просмотр вперед: убеждаемся, что за ним следует не-пробельный символ.
    # (?![.,;:!?\s]): Негативный просмотр вперед: убеждаемся, что за ним НЕ следует другой знак препинания или пробел.
    text = re.sub(r'([.,;:!?]|___ELLIPSIS___)(?=\S)(?![.,;:!?\s])', r'\1 ', text)


    # --- 4. Обработка дефисов/тире (-) ---
    # Склеиваем слова, соединенные дефисом с пробелами.
    # Пример: "слово - слово" -> "слово-слово"
    text = re.sub(r'(?<=\w)\s*-\s*(?=\w)', '-', text)

    # Удаляем висячие дефисы в начале или конце строки, если они не являются частью слова.
    # Пример: "- Слово" -> "Слово", "Слово -" -> "Слово"
    text = re.sub(r'^- ', '', text)
    text = re.sub(r' -$', '', text)

    # Исправляем паттерн "слово- ." -> "слово-."
    text = re.sub(r'(?<=\w)-([.,;:!?])', r'-\1', text)

    # --- 5. Восстановление защищенных паттернов ---
    # Возвращаем многоточия на место.
    text = text.replace('___ELLIPSIS___', '...')

    # --- 6. Финальная обрезка пробелов ---
    # Удаляем любые оставшиеся пробелы в начале и конце строки.
    text = text.strip()
        
    # if text != original_text_for_debug:
    #     print(f"DEBUG Post-Process: \n  Original: '{original_text_for_debug}'\n  Cleaned:  '{text}'")
        
    return text

# --- Основная логика ---
def main():
    print(f"Starting to process file: {INPUT_XML_FILE_TO_CLEAN}")
    try:
        parser = ET.XMLParser(remove_blank_text=True)
        print("Parsing XML file...")
        tree = ET.parse(INPUT_XML_FILE_TO_CLEAN, parser)
        root = tree.getroot()
        print("XML file parsed.")
    except FileNotFoundError:
        print(f"Error: Input file not found at {INPUT_XML_FILE_TO_CLEAN}")
        return
    except ET.XMLSyntaxError as e:
        print(f"Error: Could not parse XML file {INPUT_XML_FILE_TO_CLEAN}: {e}")
        return

    nodes_to_process = []
    print("Collecting text nodes for processing...")
    for element in root.iter():
        if not isinstance(element.tag, str) or element.tag.lower() in ["infotexts", "style"]:
            continue
        if not element.text:
            continue
        nodes_to_process.append(element)
    
    if not nodes_to_process:
        print("No text nodes found to process.")
        return

    print(f"Found {len(nodes_to_process)} text nodes. Starting cleaning process...")
    
    nodes_changed = 0

    for element in tqdm(nodes_to_process, desc="Cleaning XML", unit="node", ncols=100):
        escaped_full_text = element.text
        unescaped_full_text = unescape(escaped_full_text)
        
        parts = unescaped_full_text.split(TEXT_SEPARATOR, 1)
        
        new_unescaped_full_text = unescaped_full_text 
        changed_this_node = False

        if len(parts) == 2:
            translated_part = parts[0]
            original_part = parts[1]
            cleaned_translated_part = post_process_translation(translated_part)
            
            if cleaned_translated_part != translated_part:
                changed_this_node = True
            new_unescaped_full_text = f"{cleaned_translated_part}{TEXT_SEPARATOR}{original_part}"
        else:
            cleaned_full_text = post_process_translation(unescaped_full_text)
            if cleaned_full_text != unescaped_full_text:
                changed_this_node = True
            new_unescaped_full_text = cleaned_full_text
        
        if changed_this_node:
            nodes_changed += 1
        
        element.text = escape(new_unescaped_full_text)

    print(f"\nCleaning finished. Processed {len(nodes_to_process)} text elements.")
    print(f"Changed {nodes_changed} text elements due to cleaning.")

    print("Saving cleaned XML file...")
    try:
        output_dir = os.path.dirname(OUTPUT_XML_FILE_CLEANED)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)

        tree.write(OUTPUT_XML_FILE_CLEANED, encoding="utf-8", xml_declaration=True, pretty_print=True)
        print(f"Cleaned XML saved to: {os.path.abspath(OUTPUT_XML_FILE_CLEANED)}")
    except Exception as e:
        print(f"Error saving cleaned XML file: {e}")

if __name__ == "__main__":
    main()

Starting to process file: translation_output_final/translated_with_inline_originals.xml
Parsing XML file...
XML file parsed.
Found 7411 text nodes. Starting cleaning process...


Cleaning XML: 100%|████████████████████████████████████████| 7411/7411 [00:00<00:00, 30418.07node/s]


Cleaning finished. Processed 7411 text elements.
Changed 1409 text elements due to cleaning.
Saving cleaned XML file...
Cleaned XML saved to: E:\coding\!Jupetr\Barotrauma\translation_output_final\translated_cleaned_lxml_tqdm.xml
